Use a simple container to hold lagged infectiousness to pass between individuals...

In [60]:
import numpy as np

n = 5  # n_locations 
f = 0  # all-to-all fraction (vs. local component)

mixing = (f/n * np.ones((n, n)) + 
          (1-f) * np.eye(n))

mixing

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [61]:
x_vc = 0.5  # vectorial capacity multiplier
vc = np.ones(n) * x_vc

# TODO: incorporate mixing into vc_matrix and use in step() function??

In [62]:
lag_days = 10

transmissions = np.zeros((n, lag_days))

In [63]:
from emodlib.malaria import IntrahostComponent

individuals_per_pop = 5
pops = {i: [IntrahostComponent.create() for _ in range(individuals_per_pop)] for i in range(n)}

In [64]:
for i, pop in pops.items():
    pop[0].challenge()

In [65]:
def summary(pops):
    infected = {i: sum([ic.n_infections > 0 for ic in pop]) for i, pop in pops.items()}
    infectiousness = {i: sum([ic.infectiousness for ic in pop]) for i, pop in pops.items()}
    print('Infected: {}\nInfectiousness: {}'.format(infected, infectiousness))

In [66]:
summary(pops)

Infected: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
Infectiousness: {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0}


In [67]:
def step(pops, transmissions, updates_per_day=2):

    challenges = transmissions[:, -1].copy()  # cache final lagged values

    transmissions[:,1:] = transmissions[:, :-1]  # shift forward lagged by day

    for i, pop in pops.items():
        for ic in pop:

            for _ in range(updates_per_day):
                ic.update(dt=1.0/updates_per_day)

        n_challenges = np.random.poisson(challenges[i])
        for ix in np.random.choice(len(pop), size=n_challenges):
            pop[ix].challenge()

        infectiousness = np.mean([ic.infectiousness for ic in pop])

        transmissions[i][0] = infectiousness * vc[i]  # add today's infectiousness (#TODO: vc_matrix mixing)

In [90]:
timesteps = 10

for t in range(timesteps):
    step(pops, transmissions)

summary(pops)
# print(transmissions)


Infected: {0: 4, 1: 5, 2: 3, 3: 4, 4: 1}
Infectiousness: {0: 0.3769115204985951, 1: 0.9685992894665105, 2: 0.19582680401799735, 3: 0.13518626367658726, 4: 8.250575046986341e-05}
